# Lectura de datos desde archivo

In [ ]:
import numpy as np
import random


data = np.loadtxt('/content/drive/MyDrive/UGR/IC/Practica2/qap.datos/tai256c.dat', skiprows=1)

# Separación en matrices de flujos y distancias

In [ ]:
flow = np.int32(data[:256])
distances = np.int32(data[256:])

# Configuración de la semilla para reproducibilidad

In [ ]:
seed = 2024
random.seed(seed)
np.random.seed(seed)

# Generación de población inicial de 2000 individuos

In [ ]:
population = np.array([np.random.permutation(np.arange(256)) for _ in range(2000)])

# Función de fitness eficiente usando operaciones matriciales


In [ ]:
def fitness_pop(population, flow, distances):
    return np.sum(flow[np.newaxis, :, :] * distances[population[:,:,np.newaxis], population[:, np.newaxis, :]], axis=(1,2))


# Selección por torneo

In [ ]:
def selection_tournament(population, fitness_vals, k=3):
    selected_indices = np.array([random.choices(np.arange(len(fitness_vals)), k=k, weights=1/fitness_vals)[0] for _ in range(len(population))])
    return population[selected_indices]


# Cruce de orden (Order Crossover, OX)

In [ ]:
def crossover_order(parent1, parent2):
    n = len(parent1)
    start, end = sorted(random.sample(range(n), 2))
    child = -np.ones(n, dtype=int)
    child[start:end+1] = parent1[start:end+1]

    position = 0
    for gene in parent2:
        if gene not in child:
            while child[position] != -1:
                position += 1
            child[position] = gene
    return child

def crossover_population(population, crossover_prob=0.9):
    new_population = []
    for _ in range(len(population) // 2):
        parent1, parent2 = population[random.sample(range(len(population)), 2)]
        if random.random() < crossover_prob:
            child1 = crossover_order(parent1, parent2)
            child2 = crossover_order(parent2, parent1)
        else:
            child1, child2 = parent1.copy(), parent2.copy()
        new_population.extend([child1, child2])
    return np.array(new_population)


# Mutación por intercambio (Swap Mutation)

In [ ]:
def mutate_population(population, mutation_prob=0.3):
    for individual in population:
        if random.random() < mutation_prob:
            i, j = random.sample(range(len(individual)), 2)
            individual[i], individual[j] = individual[j], individual[i]
    return population

# Búsqueda local (2-opt limitada)

In [ ]:
def local_search_2opt_limited(permutation, flow, distances, max_iter=20):
    n = len(permutation)
    best_cost = calculate_cost(permutation, flow, distances)
    for _ in range(max_iter):
        i, j = sorted(random.sample(range(n), 2))
        new_perm = permutation.copy()
        new_perm[i:j+1] = list(reversed(new_perm[i:j+1]))
        new_cost = calculate_cost(new_perm, flow, distances)
        if new_cost < best_cost:
            permutation = new_perm
            best_cost = new_cost
    return permutation

def calculate_cost(permutation, flow, distances):
    indexed_distances = distances[permutation][:, permutation]
    cost = np.sum(flow * indexed_distances)
    return cost

# Aplicar búsqueda local a los mejores individuos

In [ ]:
def apply_2opt_to_elite(population, fitness_vals, flow, distances, elite_ratio=0.2):
    elite_size = int(len(population) * elite_ratio)
    elite_indices = np.argsort(fitness_vals)[:elite_size]
    for idx in elite_indices:
        population[idx] = local_search_2opt_limited(population[idx], flow, distances)
    return population

# Ciclo evolutivo

In [ ]:
generations = 200
crossover_prob = 0.9
mutation_prob = 0.3

best_solution = None
best_fitness = float('inf')

fitness_vals = fitness_pop(population, flow, distances)

for generation in range(generations):
    # Selección
    selected_population = selection_tournament(population, fitness_vals)

    # Cruce
    offspring = crossover_population(selected_population, crossover_prob)

    # Mutación
    population = mutate_population(offspring, mutation_prob)

    # Aplicar búsqueda local a la élite
    population = apply_2opt_to_elite(population, fitness_vals, flow, distances, elite_ratio=0.2)

    # Recalcular fitness
    fitness_vals = fitness_pop(population, flow, distances)

    # Guardar el mejor individuo
    gen_best_fitness = fitness_vals.min()
    if gen_best_fitness < best_fitness:
        best_fitness = gen_best_fitness
        best_solution = population[np.argmin(fitness_vals)]

    print(f"Generación {generation + 1}: Mejor Fitness = {best_fitness}")
    if best_fitness <= 44759294:
        print("Objetivo alcanzado, deteniendo el proceso.")
        break

print("Mejor solución encontrada:", best_solution)
print("Costo asociado:", best_fitness)


Generación 1: Mejor Fitness = 50044002
Generación 2: Mejor Fitness = 49759852
Generación 3: Mejor Fitness = 49759852
Generación 4: Mejor Fitness = 49759852
Generación 5: Mejor Fitness = 49549784
Generación 6: Mejor Fitness = 49218524
Generación 7: Mejor Fitness = 49218524
Generación 8: Mejor Fitness = 49218524
Generación 9: Mejor Fitness = 49218524
Generación 10: Mejor Fitness = 49218524
Generación 11: Mejor Fitness = 49218524
Generación 12: Mejor Fitness = 49218524
Generación 13: Mejor Fitness = 49218524
Generación 14: Mejor Fitness = 49218524
Generación 15: Mejor Fitness = 49218524
Generación 16: Mejor Fitness = 49218524
Generación 17: Mejor Fitness = 49218524
Generación 18: Mejor Fitness = 49218524
Generación 19: Mejor Fitness = 49218524
Generación 20: Mejor Fitness = 49218524
Generación 21: Mejor Fitness = 49218524
Generación 22: Mejor Fitness = 49218524
Generación 23: Mejor Fitness = 49218524
Generación 24: Mejor Fitness = 49218524
Generación 25: Mejor Fitness = 49164536
Generació